# Sweet Lift Taxi

Sweet Lift Taxi company has collected data on taxi orders at airports. Their aim is to predict the amount of taxi orders for the next hour, in order to allocate more drivers for peak hours. We will build a model with an RMSE lower than 48. 

****************

In [ ]:
# !pip install --user plotly_express

## Import Libraries

In [ ]:
# Import libraries
import pandas as pd 
import numpy as np
import plotly_express as px 
from matplotlib import pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, make_scorer
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor 
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool

In [ ]:
# read dataframe
df = pd.read_csv('datasets/taxi.csv', parse_dates=['datetime'], index_col=['datetime'])

In [ ]:
# sorting index
df.sort_index(inplace=True)

In [ ]:
# checking if index is monotonic
print(df.index.is_monotonic)

In [ ]:
# look at dataframe
df.head()

In [ ]:
# info on num orders columns
df.info()

In [ ]:
# confirming no missing values
df.isna().sum()

In [ ]:
# resample data by the hour
df = df.resample('1H').sum()

We loaded the data and then converted the dates into datetime format. We then made the datetime column our index, and sorted the index. We checked to make sure the data was free of missing values. Following that, we resampled the data by the hour. 

*************

## EDA

In [ ]:
# summary statistics on the number of orders
df.describe()

In [ ]:
# hourly number of orders
fig = px.line(df.num_orders, title='Total Hourly Number of Orders', template='ggplot2', height=600, labels={'value': 'Number of Orders'})
fig.update_xaxes(rangeslider_visible=True, 
    rangeselector=dict(
        buttons=list([
            dict(count=1, label='1m', step='month', stepmode='backward'), 
            dict(count=6, label='6m', step='month', stepmode='backward')
            ])
        )
    )
fig.show()

This is a visual of the timeseries data sampled on the hour. The y axis shows the number of taxi orders. 

In [ ]:
# distribution of orders
px.box(df.num_orders, title='Distribution of Orders', template='ggplot2', labels={'variable': 'Orders', 'value': 'count'}, height=600)

Here, we have the distribution of the number of orders. We see some outliers with values above 186. The average is 78 orders. 

In [ ]:
# Average daily orders
numbers = [6, 12, 24, 168, 720] 
for i in numbers:
    px.line(df.num_orders.rolling(i).mean(), title=f'Mean Number of Orders per {i} Hours', template='ggplot2', labels={'value': 'Number of Orders', 'datetime':'Dates'}, height=600).show()

These visuals show the number of orders resampled for 6 hours, 12 hours, per day, per week, and per month. These visualizations allow us the clearly see the trend in orders increase gradually from April to August. 

In [ ]:
# decomposed dataset
decomposed = seasonal_decompose(df)

In [ ]:
# decomposed trend 
px.line(decomposed.trend, title='Trend', template='ggplot2')

In [ ]:
# decomposed seasonality 
px.line(decomposed.seasonal, title='Seasonality', template='ggplot2')

In [ ]:
# decomposed residual 
px.line(decomposed.resid, title='Residual', template='ggplot2')

These visualizations show us the decomposed trends of the data. The trend is the same chart as the resampled daily chart. We see the seasonality chart is tight, which may be due to the short window of time given by the data. The residuals generally fluctuate around 0, but starts to show outliers in August. 

In [ ]:
# difference in number of orders
fig = px.line(df.num_orders-df.num_orders.shift(), title='Difference in Number of Orders', template='ggplot2', height=700)
fig.update_xaxes(rangeslider_visible=True, 
    rangeselector=dict(
        buttons=list([
            dict(count=1, label='1m', step='month', stepmode='backward'), 
            dict(count=6, label='6m', step='month', stepmode='backward')
            ])
        )
    )
fig.show()

Looking at the shifted difference in the number of orders, we see the values increase as time increases. These differences become more pronounced from August onward.  

**************

## Model Preparation

In [ ]:
# Making features, max lag 24 and rolling mean 24
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['hour'] = data.index.hour

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = (
        data['num_orders'].shift().rolling(rolling_mean_size).mean()
    )


make_features(df, 24, 24)

In [ ]:
# splitting dataset to train, valid, and test
train, test = train_test_split(df, shuffle=False, test_size=0.1, random_state=19)
train, valid = train_test_split(train, shuffle=False, test_size=0.11, random_state=19)

print('Train Dataset = ', ' Start : ', train.index.min(), '   End : ', train.index.max(), '   Difference : ', abs(train.index.min() - train.index.max()))
print('Valid Dataset = ', ' Start : ', valid.index.min(), '   End : ', valid.index.max(), '   Difference : ', abs(valid.index.min() - valid.index.max()))
print('Test Dataset = ', ' Start : ', test.index.min(),  '   End : ', test.index.max(), '   Difference : ', abs(test.index.min() - test.index.max()))

In [ ]:
# Dropping missing values from datasets 
train = train.dropna()
valid = valid.dropna() 
test = test.dropna()

In [ ]:
# Splitting target and features
X_train = train.drop(columns='num_orders')
y_train = train.num_orders

X_valid = valid.drop(columns='num_orders')
y_valid = valid.num_orders 

X_test = test.drop(columns='num_orders')
y_test = test.num_orders

In [ ]:
features = df.drop(columns='num_orders')
target = df.num_orders

In [ ]:
# visual of train test split
fig, ax = plt.subplots(figsize=(25,15))
train.num_orders.plot(ax=ax, label='Training Set', title='Data Train/Test Split')
valid.num_orders.plot(ax=ax, label='Valid Set', color='green')
test.num_orders.plot(ax=ax, label='Test Set')
ax.axvline('2018-07-26 08:00:00', color='black', ls='--')
ax.axvline('2018-08-13 14:00:00', color='black', ls='--')
ax.legend(['Training Set', 'Test Set'])
plt.show()


We define a function to make features, with a max lag of 24 and a rolling mean of 24. We then split the data into three parts: train, valid, and test. We will train the models with the training data, then tune the models with the validation set. The test set is reserved for evaluating the performance of the final model we choose. Since it is crucial to have adequate training data, we limited the validation and test sets to 10% of the data each. This leaves roughly 80% of the data for training. Furthermore, with time series, we can not randomly select points in our data to split, so shuffle was set to false. This gives us the correct sequence in the order of the different sets, made evident by the last figure. 

**************

## Modeling

### Linear Regression

In [ ]:
# linear regression
lr = LinearRegression() # initialize model constructor
lr.fit(X_train, y_train) # train model on training set

predictions_valid_lr = lr.predict(X_valid) # get model predictions on validation set

result = mse(y_valid, predictions_valid_lr) ** 0.5 # calculate RMSE on validation set
print("RMSE of the linear regression model on the validation set:", result)


In [ ]:
# Get the feature importances of the best model
lr_importances = lr.coef_

# Create a dataframe with the feature importances and the corresponding feature names
lr_importances_df = pd.DataFrame({'feature':X_train.columns, 'coefficients':lr.coef_})

# Sort the dataframe by importance
lr_importances_df.sort_values(by='coefficients', ascending=False, inplace=True)
lr_importances_df

The rolling mean is the coefficient with the highest value among the linear regression features. We achieve an RMSE score of 34.28 with the validation set. 

### Decision Tree

In [ ]:
# Decision Tree
best_model = None
best_result = 50
best_depth = 0
for depth in range(1, 6): # choose hyperparameter range
    dtr = DecisionTreeRegressor(random_state=19, max_depth=depth)
    dtr.fit(X_train, y_train) # train model on training set
    predictions_valid_dtr = dtr.predict(X_valid) # get model predictions on validation set
    result = mse(y_valid, predictions_valid_dtr) ** 0.5
    if result < best_result:
        best_model = dtr
        best_result = result
        best_depth = depth

print(f"RMSE of the best model on the validation set (max_depth = {best_depth}): {best_result}")

In [ ]:
# Get the feature importances of the best model
dtr_importances = dtr.feature_importances_

# Create a dataframe with the feature importances and the corresponding feature names
dtr_importances_df = pd.DataFrame({'feature':X_train.columns, 'importance':dtr.feature_importances_})

# Sort the dataframe by importance
dtr_importances_df.sort_values(by='importance', ascending=False, inplace=True)

In [ ]:
# top 10 feature importances
px.pie(dtr_importances_df.head(10), names='feature', values='importance', title='Top 10 Feature Importance for Decision Tree Regression', template='ggplot2', hole=0.2)

The lag 24 feature is the most important in the decesion tree regression. We achieve an RMSE score of 38.54 with the validation set. 

### Random Forest

In [ ]:
# Random Forest 
best_model = None
best_result = 50
best_est = 0
best_depth = 0
for est in range(600, 601):
    for depth in range (100, 101):
        rf = RandomForestRegressor(random_state=19, n_estimators=est, max_depth=depth)
        rf.fit(X_train, y_train) # train model on training set
        predictions_valid = rf.predict(X_valid) # get model predictions on validation set
        result = mse(y_valid, predictions_valid) ** 0.5 # calculate RMSE on validation set
        if result < best_result:
            best_model = rf
            best_result = result
            best_est = est
            best_depth = depth
            
print("RMSE of the best model on the validation set:", best_result, "n_estimators:", best_est, "best_depth:", depth)

In [ ]:
# Get the feature importances of the best model
rf_importances = best_model.feature_importances_

# Create a dataframe with the feature importances and the corresponding feature names
rf_importances_df = pd.DataFrame({'feature':X_train.columns, 'importance':rf.feature_importances_})

# Sort the dataframe by importance
rf_importances_df.sort_values(by='importance', ascending=False, inplace=True)

In [ ]:
# top 10 feature importances
px.pie(rf_importances_df.head(10), names='feature', values='importance', title='Top 10 Feature Importance for Random Forest Regression', template='ggplot2', hole=0.2)

The Lag 24 feature has the greatest importance in the random forest model. The RMSE score is 32.01. 

### Ada Boost

In [ ]:
# ADA Boost
regr = AdaBoostRegressor(random_state=19, n_estimators=100)
regr.fit(X_train, y_train)  

predictions_valid_regr = regr.predict(X_valid) # get model predictions on validation set

result = mse(y_valid, predictions_valid_regr) ** 0.5 # calculate RMSE on validation set
print("RMSE of the ada boost regression model on the validation set:", result)

In [ ]:
# table of feature importance
regr_imp = [t for t in zip(features, regr.feature_importances_)]
regr_imp_df = pd.DataFrame(regr_imp, columns=['feature', 'varimp'])
regr_imp_df = regr_imp_df.sort_values('varimp', ascending=False)


In [ ]:
# top 10 feature importances
px.pie(regr_imp_df.head(10), names='feature', values='varimp', title='Top 10 Feature Importance for Ada Boost Regresion', hole=.2, template='ggplot2')

The lag 24 feature is the most important, followed by the lag 1, among the Ada Boost model. The RMSE score is 34.89.

### Gradient Boosting

In [ ]:
# Gradient Boost
gbr = GradientBoostingRegressor(random_state=19, learning_rate=0.2, n_estimators=1000, verbose=100, max_depth=3)
gbr.fit(X_train, y_train)

predictions_valid_gbr = gbr.predict(X_valid)

result = mse(y_valid, predictions_valid_gbr) ** 0.5 # calculate RMSE on validation set
print("RMSE of the gradient boosting model on the validation set:", result)

In [ ]:
# table of feature importance
gbr_imp = [t for t in zip(features, gbr.feature_importances_)]
gbr_imp_df = pd.DataFrame(gbr_imp, columns=['feature', 'varimp'])
gbr_imp_df = gbr_imp_df.sort_values('varimp', ascending=False)

In [ ]:
# top 10 feature importances
px.pie(gbr_imp_df.head(10), names='feature', values='varimp', title='Top 10 Feature Importance for Gradient Boosting', hole=.2, template='ggplot2')

The lag 24 feature is the most important, followed by the lag 1, among the Gradient Boost model. The RMSE score is 35.12.

### XG boost

In [ ]:
# XGB 
xgbr = XGBRegressor(learning_rate=0.09, n_estimators=800, eval_metric='rmse', random_state=19, max_depth=6, early_stopping_rounds=500)

xgbr.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=20)

# Make predictions on the test set
predictions_xgbr = xgbr.predict(X_valid)

result = mse(y_valid, predictions_xgbr) ** 0.5 # calculate RMSE on validation set
print()
print("RMSE of the xgbm model on the validation set:", result)

In [ ]:
# table of feature importance
xgbr_imp = [t for t in zip(features, xgbr.feature_importances_)]
xgbr_imp_df = pd.DataFrame(xgbr_imp, columns=['feature', 'varimp'])
xgbr_imp_df = xgbr_imp_df.sort_values('varimp', ascending=False)

In [ ]:
# top 10 feature importances
px.pie(xgbr_imp_df.head(10), names='feature', values='varimp', title='Top 10 Feature Importance for XG Boost', hole=.2, template='ggplot2')

The lag 24 feature is the most important, followed by the lag 1, among the XG Boost model. The RMSE score is 31.69.

### Light GBM

In [ ]:
# LGBM

# Create a LightGBM dataset
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

# Define the parameters for the LightGBM model
params = {
    'objective': 'regression',
    'metric': 'root_mean_squared_error',
    'boosting_type': 'gbdt',
    'random_state': 19
}

# Train the LightGBM model
lgbm = lgb.train(params, lgb_train, valid_sets=lgb_valid, num_boost_round=500, early_stopping_rounds=50)

# Make predictions on the test set
predictions_valid_lgbm = lgbm.predict(X_valid)

result = mse(y_valid, predictions_valid_lgbm) ** 0.5 # calculate RMSE on validation set
print()
print("RMSE of the lgbm model on the validation set:", result)

In [ ]:
# Get the feature importances of the trained model
lgbm_importances = lgbm.feature_importance()

# Create a dataframe with the feature importances and the corresponding feature names
lgbm_importances_df = pd.DataFrame({'feature':X_train.columns, 'importance':lgbm_importances})

# Sort the dataframe by importance
lgbm_importances_df.sort_values(by='importance', ascending=False, inplace=True)


In [ ]:
# top 10 feature importances
px.pie(lgbm_importances_df.head(10), names='feature', values='importance', title='Top 10 Feature Importance for Light GBM Regression', hole=.2, template='ggplot2')

The lag 24 feature is the most important, followed by the lag 1, 3, and 17,  among the light GB model. The RMSE score is 31.51.

### Catboost

In [ ]:
# catboost

catb = CatBoostRegressor(task_type='GPU', loss_function='RMSE', eval_metric='RMSE', iterations=2000, random_seed=19, early_stopping_rounds=500)

catb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=100, use_best_model=True, plot=True)

# Make predictions on the test set
predictions_valid_catb = catb.predict(X_valid)

result = mse(y_valid, predictions_valid_catb) ** 0.5 # calculate RMSE on validation set
print()
print("Catboost model on the test set: ")
catb.best_score_

In [ ]:
# Get the feature importances of the trained model
catb_importances = catb.feature_importances_

# Create a dataframe with the feature importances and the corresponding feature names
catb_importances_df = pd.DataFrame({'feature':X_train.columns, 'importance':catb_importances})

# Sort the dataframe by importance
catb_importances_df.sort_values(by='importance', ascending=False, inplace=True)

In [ ]:
# top 10 feature importances
px.pie(catb_importances_df.head(10), names='feature', values='importance', title='Top 10 Feature Importance for Catboost Regression', hole=.2, template='ggplot2')

The lag 24 feature is the most important, followed by the lag 1, among the catboost model. The RMSE score is 30.42.

### Voting Regressor

In [ ]:
# Training classifiers
# reg1 = RandomForestRegressor(random_state=12345, n_estimators=90, max_depth=100)
# reg2 = GradientBoostingRegressor(random_state=19, learning_rate=0.2, n_estimators=1000, verbose=1, max_depth=3)
reg3 = XGBRegressor(learning_rate=0.09, n_estimators=800, eval_metric='rmse', random_state=19, max_depth=6) #, early_stopping_rounds=500)
reg4 = lgb.LGBMRegressor(objective='regression', metric='root_mean_squared_error', boosting_type='gbdt', random_state=19) #, early_stopping_rounds=50)
reg5 = CatBoostRegressor(task_type='GPU', loss_function='RMSE', eval_metric='RMSE', iterations=2000, random_seed=19) #, early_stopping_rounds=500)

ereg = VotingRegressor(estimators=[#('rf', reg1), 
                                #('gbr', reg2), 
                                ('xgb', reg3), 
                                ('lgb', reg4), 
                                ('cat', reg5)], 
                                verbose=1)
ereg = ereg.fit(X_train, y_train)

# Make predictions on the test set
predictions_valid_ereg = ereg.predict(X_valid)

result = mse(y_valid, predictions_valid_ereg) ** 0.5 # calculate RMSE on validation set
print()
print("voting regressor model on the valid set: ", result)

The best model we have is the voting regressor. This model combines the three best performing models and parameters: XG boost, Light GB, and Catboost. The RMSE score is 30.78 with the validation set. 

## Comparing Model Scores

In [ ]:
# making model scores dataframe
model_scores = pd.DataFrame({'Linear Regression': 34.28, 'Decision Tree': 38.54, 'Random Forest': 32.01, 'Ada Boost': 34.89, 'Gradient Boost': 35.11, 'XG Boost': 31.69, 'Light GBM': 31.51,
             'Catboost': 30.42, 'Voting Regressor': 30.78}, index={'RMSE'})
model_scores = model_scores.T


In [ ]:
# Model RMSE scores
px.scatter(model_scores, title='Model RMSE Scores', template='ggplot2', color=model_scores.index, size='RMSE', y='RMSE', size_max=30, labels={'index': 'Model'})

This figure compares the RMSE scores of the various models. The three best performing models are the Gradient boost, XG boost, and Catboost. Finally, the Voting regressor achieved a great score.  

## Final Model

In [ ]:
# combine validation set with training set
X_full = pd.concat([X_train, X_valid])
y_full = pd.concat([y_train, y_valid])

In [ ]:
# Training classifiers
# reg1 = RandomForestRegressor(random_state=12345, n_estimators=90, max_depth=100)
# reg2 = GradientBoostingRegressor(random_state=19, learning_rate=0.2, n_estimators=1000, verbose=1, max_depth=3)
reg3 = XGBRegressor(learning_rate=0.09, n_estimators=800, eval_metric='rmse', random_state=19, max_depth=6) #, early_stopping_rounds=500)
reg4 = lgb.LGBMRegressor(objective='regression', metric='root_mean_squared_error', boosting_type='gbdt', random_state=19) #, early_stopping_rounds=50)
reg5 = CatBoostRegressor(task_type='GPU', loss_function='RMSE', eval_metric='RMSE', iterations=2000, random_seed=19) #, early_stopping_rounds=500)

final = VotingRegressor(estimators=[#('rf', reg1), 
                                #('gbr', reg2), 
                                ('xgb', reg3), 
                                ('lgb', reg4), 
                                ('cat', reg5)], 
                                verbose=1)
final = final.fit(X_full, y_full)

# Make predictions on the test set
final_predictions = final.predict(X_test)

result = mse(y_test, final_predictions) ** 0.5 # calculate RMSE on validation set
print()
print("voting regressor model on the test set: ", result)

The final model we chose was the voting regressor. We achieved a RMSE score of 41.24 with the test set. This model performs better than the required RMSE score of 48. Therefore, we have accurately predicted the future number of orders. 

### Overall Conclusions

Overall, we succeeded in providing a model for Sweet Lift Taxi to predict the number of orders of the next hour. The target metric for our model was an RMSE score under 48. Our final model was a voting regressor, with a final RMSE of 46.47 with the test data set. Therefore, Sweet Lift can accommodate drivers with a model that accurately predicts future number of orders.  